In [1]:
import numpy as np
import sys
import os
import pandas as pd
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from levenberg_marquadt import levenberg_marquadt
from normalize import  MinMaxNormalizer
from loss_fn_tarefa4 import make_mse_loss_for_neuron
from activations_fn import tanh_derivative
from network import neuron
from plots.plots_tarefa4 import plot_original_surface, plot_predicted_surface_comparison

pd.set_option('display.float_format', '{:.5f}'.format)

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho4dados.xlsx')

df.head()

,x1,x2,y
0,-5.00000,-3.00000,-0.91722
1,-4.89899,-2.93939,-0.93968
2,-4.79798,-2.87879,-0.91822
3,-4.69697,-2.81818,-0.90445
4,-4.59596,-2.75758,-0.93496


### Mostrar superfície dos dados originais

In [3]:
# Extrair os dados
x1_data = df['x1'].values
x2_data = df['x2'].values
y_data = df['y'].values

fig_surface = plot_original_surface(x1_data, x2_data, y_data)

## Calcular as funções de perda

### Funções utilizadas para os cálculos

In [4]:
def make_residuals_fn(X1: np.ndarray, X2: np.ndarray, Y: np.ndarray, activation_fn=np.tanh):
    def residuals_fn(w: np.ndarray) -> np.ndarray:
        predictions = neuron(X1, X2, weights=w, activation_fn=activation_fn)
        return Y - predictions
    return residuals_fn


def make_jacobian_fn(X1, X2, activation_fn=np.tanh, activation_deriv=tanh_derivative):
    """
    Jacobiana genérica: derivada dos resíduos em relação aos pesos.
    Funciona para qualquer função de ativação.
    """
    def jacobian_fn(w):
        X1_ = np.atleast_1d(X1)
        X2_ = np.atleast_1d(X2)
        X = np.stack([X1_, X2_, np.ones_like(X1_)], axis=1)
        z = X @ w
        phi_prime = activation_deriv(z)
        J = -phi_prime[:, np.newaxis] * X  # cada linha: -φ'(z_i) * [x1, x2, 1]
        return J
    return jacobian_fn

### Pré-processamento dos dados

In [5]:

features = df[['x1', 'x2']]
y = df['y']

# Criar os objetos para Padronização
scaler = MinMaxNormalizer(-1, 1)
scaler_y = MinMaxNormalizer(-1, 1)

# Cria as cópias dos dados para padronizar
scaled_features = features.copy()
scaled_y = y.copy()

# Ajusta os padronizadores aos dados
scaler.fit(scaled_features)
scaler_y.fit(scaled_y.to_frame())

# Padroniza os dados
scaled_features = scaler.normalize(scaled_features)
scaled_y = scaler_y.normalize(scaled_y.to_frame()).squeeze()

# initial_weights = np.random.randn(features.shape[1] + 1)  # 2 entradas + 1 bias
initial_weights = np.zeros(features.shape[1] + 1)  # 2 entradas + 1 bias
n_iterations = 10000
tolerance = 1e-6
alpha = 1e-3

### Rodar os experimentos

In [ ]:
dict_results = {}

x1_scaled = scaled_features['x1'].values
x2_scaled = scaled_features['x2'].values
y_scaled_data = scaled_y.values

# Função de custo e gradiente
loss_function, grad_loss_function = make_mse_loss_for_neuron(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)

# Função de residuos e jacobiana
residuals_fn = make_residuals_fn(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)
jacobian_fn = make_jacobian_fn(x1_scaled, x2_scaled, activation_fn=np.tanh, activation_deriv=tanh_derivative)

# Treinar o modelo com Levenberg-Marquadt
weights, losses, n_iters = levenberg_marquadt(
    initial_weights, residuals_fn, loss_function, jacobian_fn,
    alpha=alpha, alpha_variability=10, max_iter=n_iterations,
    tolerance=tolerance, stopping_criteria=[1, 3]
)

# Desnormalizar os pesos finais
final_raw_weights = weights[-1].copy()
final_denorm_weights = scaler.desnormalize_weights(weights[-1])

# Usar o neurônio com pesos desnormalizados
y_hat = neuron(x1_data, x2_data, weights=final_denorm_weights, activation_fn=np.tanh)

# Cálculo das métricas finais
mse_final = np.mean((y - y_hat) ** 2)
rmse_final = np.sqrt(mse_final)
mae_final = np.mean(np.abs(y - y_hat))

dict_results = {
    'Feature_Set': "MinMax(-1,1)",
    'Loss_Function': "MSE",
    'Initial_Weights': str([f"{w:.5f}" for w in initial_weights.tolist()]),
    'Raw_Weights': str([f"{w:.5f}" for w in final_raw_weights.tolist()]),
    'Final_Weights': str([f"{w:.5f}" for w in final_denorm_weights.tolist()]),
    'Final_Loss': losses[-1],
    'MSE_Final': mse_final,
    'RMSE_Final': rmse_final,
    'MAE_Final': mae_final,
    'Iterations': n_iters
}

# Criar DataFrame com um único registro
df_result = pd.DataFrame([dict_results]) 
df_result

,Feature_Set,Loss_Function,Initial_Weights,Raw_Weights,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,"MinMax(-1,1)",MSE,"['0.00000', '0.00000', '0.00000']","['4.60081', '4.60081', '2.07464']","['0.92016', '1.53360', '2.07464']",0.00630,0.00806,0.08975,0.06793,8


## Resultados

In [7]:
# Criar grade para superfície prevista
x1_range = np.linspace(min(x1_data), max(x1_data), 30)
x2_range = np.linspace(min(x2_data), max(x2_data), 30)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
y_grid = np.zeros_like(x1_grid)

fig_surface = plot_predicted_surface_comparison(
    fig_surface,
    x1_data, x2_data, y_data, y_hat,
    final_denorm_weights,
    neuron_fn=neuron,
    mse_final=mse_final, rmse_final=rmse_final, mae_final=mae_final
)